# Detection 02 — High-Risk Email Timing

## Phase
Phase 4 — Detection Engineering

## Objective
Identify emails delivered to sensitive users during unusual or off-hours periods.

This models scenarios where attackers target privileged users outside normal working patterns.


In [1]:
import pandas as pd
from pathlib import Path


In [2]:
PROJECT_ROOT = Path(r"D:\soc-dashboard-suite-main\soc-dashboard-suite-main")

INPUT_PATH = PROJECT_ROOT / "data" / "enriched" / "email_with_temporal_context.csv"
ALERT_OUTPUT_PATH = PROJECT_ROOT / "data" / "enriched" / "alerts_high_risk_timing.csv"

email_df = pd.read_csv(INPUT_PATH, parse_dates=["event_time"])


In [3]:
high_risk_users = email_df["user_role"].isin(["admin", "executive"])
risky_time = email_df["time_behavior"].isin(["off_hours", "unusual_hour"])

alerts_df = email_df[high_risk_users & risky_time].copy()

len(alerts_df)


37772

In [4]:
alerts_df["detection_id"] = "DET_02_HIGH_RISK_EMAIL_TIMING"

alerts_df["severity"] = alerts_df["user_role"].map({
    "admin": "high",
    "executive": "high"
})

alerts_df["alert_reason"] = "Sensitive user received email at unusual or off-hours time"


In [5]:
alert_fields = [
    "event_time",
    "sender_email",
    "sender_domain",
    "recipient_email",
    "user_role",
    "time_behavior",
    "domain_rarity",
    "is_first_seen_day",
    "severity",
    "alert_reason",
    "detection_id"
]

alerts_df = alerts_df[alert_fields]
alerts_df.head()


,event_time,sender_email,sender_domain,recipient_email,user_role,time_behavior,domain_rarity,is_first_seen_day,severity,alert_reason,detection_id
7,2000-07-14 06:59:00-07:00,phillip.allen@enron.com,enron.com,joyce.teixeira@enron.com,executive,off_hours,common,False,high,Sensitive user received email at unusual or of...,DET_02_HIGH_RISK_EMAIL_TIMING
11,2000-10-13 06:45:00-07:00,phillip.allen@enron.com,enron.com,stagecoachmama@hotmail.com,admin,off_hours,common,False,high,Sensitive user received email at unusual or of...,DET_02_HIGH_RISK_EMAIL_TIMING
40,2000-09-20 06:34:00-07:00,phillip.allen@enron.com,enron.com,cbpres@austin.rr.com,admin,off_hours,common,False,high,Sensitive user received email at unusual or of...,DET_02_HIGH_RISK_EMAIL_TIMING
42,2000-09-19 07:26:00-07:00,phillip.allen@enron.com,enron.com,cbpres@austin.rr.com,admin,off_hours,common,False,high,Sensitive user received email at unusual or of...,DET_02_HIGH_RISK_EMAIL_TIMING
45,2000-09-13 06:02:00-07:00,phillip.allen@enron.com,enron.com,stagecoachmama@hotmail.com,admin,off_hours,common,False,high,Sensitive user received email at unusual or of...,DET_02_HIGH_RISK_EMAIL_TIMING


In [6]:
alerts_df.to_csv(ALERT_OUTPUT_PATH, index=False)
print("Saved alerts to:", ALERT_OUTPUT_PATH)


Saved alerts to: D:\soc-dashboard-suite-main\soc-dashboard-suite-main\data\enriched\alerts_high_risk_timing.csv


In [7]:
alerts_df["severity"].value_counts()


severity
high    37772
Name: count, dtype: int64

In [8]:
len(alerts_df) / len(email_df)


0.0762217639248195